- 입력 : 서울시 공공데이터에서 제공되는 서울시 전체 버스정류장의 위도·경도 좌표 데이터
- 출력 : 서울시 광진구의 버스정류장 데이터 (버스정류장 ID, 위도·경도, 행정동)

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 데이터 불러오기
station = pd.read_csv('./data/seoul_bus_station.csv', encoding='cp949')
boarding = pd.read_csv('./data/2021_boarding_timely.csv', encoding='cp949')

C:\Users\cjkang\AppData\Local\Temp\ipykernel_18680\1783597532.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  boarding = pd.read_csv('./data/2021_boarding_timely.csv', encoding='cp949')


In [10]:
# 필요 없는 컬럼 제거
drop_cols = ['정류장_명칭', '정류장_유형', '정류장_번호']
station = station.drop(drop_cols, axis=1)

station.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부
0,717104358,126.817473,37.452473,미설치
1,500000288,126.691556,37.321742,미설치
2,500000287,126.691728,37.321942,미설치
3,500000286,126.710485,37.319934,미설치
4,500000285,126.707130,37.320566,미설치


In [14]:
# 결측치 제거
station = station.dropna()
station = station.reset_index(drop=True)
station.tail()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부
70827,100000005,127.001744,37.586243,설치
70828,100000004,126.987613,37.568579,설치
70829,100000003,126.998340,37.582671,설치
70830,100000002,126.996566,37.579183,설치
70831,100000001,126.987750,37.569765,설치


## Kakao REST API

### 테스트

In [29]:
# API 문서 : URL
API_KEY = 'YOUR_KEY'
latitude = station.loc[0, '위도']
longitude = station.loc[0, '경도']
url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
headers = {
    'Authorization' : f'KakaoAK {API_KEY}'
}
params = {
    'x' : latitude,
    'y' : longitude,
    'input_coord' : 'WGS84'
}

In [30]:
# request(url, key, coord) -> response : json(str)
response = requests.get(url, params=params, headers=headers)
response

<Response [200]>

In [35]:
# json(str) -> dict -> hgd
json = response.json()
json['documents'][1]

{'region_type': 'H',
 'code': '4139051000',
 'address_name': '경기도 시흥시 대야동',
 'region_1depth_name': '경기도',
 'region_2depth_name': '시흥시',
 'region_3depth_name': '대야동',
 'region_4depth_name': '',
 'x': 126.78853828587953,
 'y': 37.44304606566207}

In [58]:
# 함수로 만들기

def get_hgd(x, y):
    API_KEY = 'YOUR_KEY'
    URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    
    headers = {
    'Authorization' : f'KakaoAK {API_KEY}'
}
    params = {
        'x' : x,
        'y' : y,
        'input_coord' : 'WGS84'
    }
    
    response = requests.get(URL, params=params, headers=headers)
    json = response.json()
    try: 
        result = (json['documents'][1]['region_2depth_name'], json['documents'][1]['region_3depth_name'])
    except:
        result = ('', '')
    return result

In [47]:
# 구 이름과 행정동 데이터가 들어갈 자리 마련
station_hgd = station.copy()
station_hgd['구이름'] = ''
station_hgd['행정동'] = ''
station_hgd.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
0,717104358,126.817473,37.452473,미설치,,
1,500000288,126.691556,37.321742,미설치,,
2,500000287,126.691728,37.321942,미설치,,
3,500000286,126.710485,37.319934,미설치,,
4,500000285,126.707130,37.320566,미설치,,


In [59]:
# 함수 실행 및 데이터 저장

for i in range(len(station_hgd)):
    if station_hgd.loc[i, '구이름'] != "":
        continue
    latitude = station_hgd.loc[i, '위도']
    longitude = station_hgd.loc[i, '경도']
    
    result = get_hgd(latitude, longitude)
    
    station_hgd.loc[i, '구이름'], station_hgd.loc[i, '행정동'] = result

station_hgd.tail()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
70827,100000005,127.001744,37.586243,설치,종로구,혜화동
70828,100000004,126.987613,37.568579,설치,종로구,종로1.2.3.4가동
70829,100000003,126.998340,37.582671,설치,종로구,혜화동
70830,100000002,126.996566,37.579183,설치,종로구,종로1.2.3.4가동
70831,100000001,126.987750,37.569765,설치,종로구,종로1.2.3.4가동


In [60]:
# 서울시 전체 데이터 저장
station_hgd.to_csv('./data/seoul_bus_station_hgd.csv', sep=',')

## 광진구만 추출

In [66]:
station_gwangjin = station_hgd.loc[station_hgd['구이름'] == '광진구']
station_gwangjin = station_gwangjin.rename(columns={"정류장_ID":"표준버스정류장ID"})
station_gwangjin.head()

,정류장_ID,위도,경도,버스도착정보안내기_설치_여부,구이름,행정동
1752,277103552,127.067091,37.531727,미설치,광진구,자양3동
1753,277103551,127.066757,37.531816,미설치,광진구,자양4동
1793,277103511,127.059683,37.533713,미설치,광진구,자양4동
1794,277103510,127.088280,37.528064,미설치,광진구,자양2동
1795,277103509,127.088549,37.528812,미설치,광진구,자양2동


In [67]:
station_gwangjin.to_csv('./data/gwangjin_bus_station.csv', index=False)